# Show difference in counts and Log2FoldChange over timepoints.

In [2]:
import scanpy as sc
import decoupler as dc
import numpy as np
import pandas as pd
import anndata as ad
from anndata import AnnData

In [16]:
metadata = pd.read_csv('../data/2023_MAFA_MetaData.txt', index_col=0, sep='\t')
metadata

,Sample_Description,Treatment,Genotype,TimePoint,Batch,DOX,GroupName
SampleID,,,,,,,
MAFAWT_001,MAFAWT,MAFA,WT,T0,1,N,MAFA_WT_T0
MAFAWT_002,MAFAWT,MAFA,WT,T0,1,N,MAFA_WT_T0
MAFAWT_003,MAFAWT,MAFA,WT,T0,1,N,MAFA_WT_T0
MAFAWT_004,MAFAWT,MAFA,WT,T0,1,N,MAFA_WT_T0
MAFAWT_005,MAFAWT,MAFA,WT,T2,1,Y,MAFA_WT_T2
MAFAWT_006,MAFAWT,MAFA,WT,T2,1,Y,MAFA_WT_T2
MAFAWT_007,MAFAWT,MAFA,WT,T2,1,Y,MAFA_WT_T2
MAFAWT_008,MAFAWT,MAFA,WT,T2,1,Y,MAFA_WT_T2
MAFAWT_009,MAFAWT,MAFA,WT,T1,1,Y,MAFA_WT_T1


In [15]:
# Read raw data and process it
adata = pd.read_csv('../data/MAFA_WT_gene_count.txt', index_col=20, sep='\t').T
adata
#subset to Genes of interest
goi = pd.read_csv("../data/table_comparison_all_GOI.csv")
subset_GOI = adata.loc[:, adata.columns.isin(goi['WT_GOI'])]
subset_GOI


gene_name,NEUROD1,NKX6-1,MAFA,PDX1,NR3C1,STAT3,GLIS3,ASCL2,MLXIP,AHR,...,MXI1,NR1D1,MITF,E2F2,NR1H4,MEF2C,WWTR1,REST,FHL2,HEYL
MAFAWT_001,10346,4195,1024,3915,4757,3058,1645,2387,1578,1398,...,172,256,66,124,105,192,23,36,109,26
MAFAWT_002,11223,6865,354,4070,6623,3180,2532,2626,1792,2713,...,203,194,57,149,69,122,18,28,74,17
MAFAWT_003,13854,9178,338,4523,6691,3128,2057,2060,1838,2961,...,257,208,82,129,136,72,28,30,59,17
MAFAWT_004,13695,8226,232,4109,8217,3590,2167,2291,1736,2918,...,235,236,121,110,173,183,35,31,77,21
MAFAWT_005,5503,16654,26618,8996,1469,6067,5033,5111,3900,856,...,350,337,468,28,52,35,93,128,17,56
MAFAWT_006,3045,11993,37529,6658,1376,5741,4917,4005,3503,844,...,340,331,529,26,17,47,150,229,11,57
MAFAWT_007,6441,16583,22361,10279,2738,7488,5341,4172,4056,1173,...,344,310,506,103,59,51,100,115,24,62
MAFAWT_008,5672,15588,24878,8627,2026,7886,4812,3915,3773,966,...,347,420,532,32,79,40,203,94,16,64
MAFAWT_009,6221,12168,18225,7001,2623,4418,3821,3462,3090,1452,...,184,176,262,95,45,39,67,65,34,21
MAFAWT_010,6365,12927,15164,6532,3591,3916,3568,2538,3048,1558,...,227,219,271,90,45,36,87,85,22,28


In [5]:
# Read in LFC + avg basemean from tables
# subset results df by list of genes from veronica
goi_t01 = pd.read_csv("../figures/direct/WT_T0_T1/goi_timepoints_01.csv")
goi_t02 = pd.read_csv("../figures/direct/WT_T0_T2/goi_timepoints_02.csv")
goi_t02

,gene_name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,NEUROD1,8770.665000,-1.173115,0.249230,-5.299530,1.161010e-07,9.622984e-07
1,NKX6-1,11026.673000,0.971090,0.222495,4.681257,2.851212e-06,1.876550e-05
2,MAFA,13793.121000,5.819076,0.505775,11.154064,6.840892e-29,3.725446e-27
3,PDX1,6344.270000,0.931824,0.162874,6.095810,1.088844e-09,1.175749e-08
4,NR3C1,4277.753400,-1.782839,0.242267,-7.673367,1.675389e-14,3.034003e-13
5,STAT3,4955.084500,0.966911,0.118007,8.438477,3.214993e-17,7.393013e-16
6,GLIS3,3532.353500,1.134861,0.178943,6.623704,3.503067e-11,4.403648e-10
7,ASCL2,3305.844200,0.755839,0.173911,4.601085,4.202949e-06,2.680824e-05
8,MLXIP,2745.727800,1.047100,0.108075,9.831188,8.263821e-23,2.996628e-21
9,AHR,1742.569000,-1.358080,0.273869,-5.327359,9.965100e-08,8.328173e-07


In [37]:
#Data wrangling to get into right format
goi_t01.loc[goi_t01['gene_name'] == 'MAFA', ['gene_name', 'baseMean', 'log2FoldChange']]
goi_t02.loc[goi_t02['gene_name'] == 'MAFA', ['gene_name', 'baseMean', 'log2FoldChange']]
#subset_GOI.loc[subset_GOI.columns =='MAFA']
subset_GOI['MAFA']


MAFAWT_001     1024
MAFAWT_002      354
MAFAWT_003      338
MAFAWT_004      232
MAFAWT_005    26618
MAFAWT_006    37529
MAFAWT_007    22361
MAFAWT_008    24878
MAFAWT_009    18225
MAFAWT_010    15164
MAFAWT_011    16847
MAFAWT_012      280
MAFAWT_013      233
MAFAWT_014      157
MAFAWT_015      366
MAFAWT_016      140
MAFAWT_017      172
MAFAWT_018      375
MAFAWT_019      227
MAFAWT_020      267
Name: MAFA, dtype: int64

In [32]:
#plot for each gene
# X = T01, T02
# Y = (average LFC), avg basemean + jitter from baseMean from individual sample values?




# plt.figure(figsize=(8, 6))
# plt.bar(df['gene_name'], df['log2FoldChange'])
# plt.xlabel('Gene Name')
# plt.ylabel('Log2 Fold Change')
# plt.title('Log2 Fold Change Bar Plot')
# plt.xticks(rotation=90)
# plt.show()



#for gene in subset_GOI.columns:
    





,gene_name,baseMean,log2FoldChange
2,MAFA,8003.367,5.225458
